In [1]:
import matplotlib.pyplot as plt 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import numpy as np
from pandas import datetime
from random import random
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.ar_model import AR
import warnings
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import math
warnings.filterwarnings('ignore')
data = pd.read_csv('/Users/alket/Desktop/dati/new_data_backfill_forwfill.csv',index_col = 0)

gbc = data.groupby(by = data['cell_num'])
cell_1 = gbc.get_group('486-1258')
cell_2 = gbc.get_group('498-1268')
series1 = cell_1['nr_people'].tolist()


In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
         # find the end of this pattern
         end_ix = i + n_steps_in
         out_end_ix = end_ix + n_steps_out
         # check if we are beyond the sequence
         if out_end_ix > len(sequence):
             break
         # gather input and output parts of the pattern
         seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
         X.append(seq_x)
         y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
def get_AR_prediction(X, st_in, st_out):
    
    # split into samples
    X_data, y_data = split_sequence(X, st_in, st_out)    
    tot_predictions = []
    
    for i in range(len(X_data)):
        model = AR(X_data[i])
        model_fit = model.fit()
        window = model_fit.k_ar
        #window = 24
        #print(window)
        coef = model_fit.params

        # walk forward over time steps in test
        history = X_data[i][len(X_data[i])-window:]
        history = [history[i] for i in range(len(history))]
        predictions = list()
        for t in range(len(y_data[i])):
            length = len(history)
            lag = [history[i] for i in range(length-window,length)]
            yhat = coef[0]
            
            for d in range(window):
                yhat += coef[d+1] * lag[window-d-1]
            
            #predictions.append(yhat)
            history.append(y_data[i][0])
            predictions.append(yhat)
            
            
        #print(predictions)
        predictions = np.asarray(predictions)
        tot_predictions.append(predictions)
    return tot_predictions

In [4]:
dict2data = {}
dict2MAPE = {}
dict2RMSE = {}
counter = 0
step_in, step_out = 2400, 3

for i, k in gbc:

    cell = i
    print(counter)
    counter +=1
    #if counter > 3: break
    k = k.iloc[::4,:]
    cell_data_i = k['nr_people'].values
    predicted = get_AR_prediction(cell_data_i, step_in, step_out)
    
    X_data, y_data = split_sequence(cell_data_i, step_in, step_out)
    print("len ", len(y_data))
    difference = abs(predicted - y_data)
    
    pow_err = np.power(difference, 2)
    rmse = math.sqrt(np.mean(pow_err))
    
    # collect data 2 dictionary
    minimum = np.amin(difference)   
    per75 = np.percentile(difference, 75)
    per50 = np.percentile(difference, 50)
    per25 = np.percentile(difference, 25)
    maximum = np.amax(difference)
    l5i = [minimum, per25, per50, per75, maximum]
    dict2data[cell] = l5i
    dict2RMSE[cell] = rmse
    MAPE = np.mean(np.divide(difference, y_data)* 100)
    dict2MAPE[cell] = MAPE
    print(rmse, MAPE, per50)
    
with open('MAE_error_data_4_Autoregression_3_StepForecast.csv', 'w') as f:
    for key, value in dict2data.items():
        f.write('%s:%s\n' % (key, value))    
        
with open('MAPE_error_data_4_Autoregression_3_StepForecast.csv', 'w') as f:
    for key, value in dict2MAPE.items():
        f.write('%s:%s\n' % (key, value))   
        
with open('RMSE_error_data_4_Autoregression_3_StepForecast.csv', 'w') as f:
    for key, value in dict2RMSE.items():
        f.write('%s:%s\n' % (key, value))          

0
len  550
3.0329937364970356 5.512007785295931 1.7417386992583719
1
len  550
3.438864335460787 5.340325590384927 1.9046351742221042
2
len  550
3.736436610216288 5.503065310060831 2.034739257150676
3
len  550
3.8491616971136784 5.440417868940856 2.089114656475818
4
len  550
3.8889132223000864 5.441023813847703 2.1237424188891083
5
len  550
3.1859412027551337 5.10904169466862 1.780642051266092
6
len  550
2.454553407605872 5.544055486933944 1.5037330861674452
7
len  550
2.441519360497029 5.45836058601229 1.4558025865485806
8
len  550
2.4472967412128184 5.480647776960962 1.5073516184095652
9
len  550
2.4445424083604848 5.45889753979825 1.4954442567315063
10
len  550
2.449085363220563 5.482427386129083 1.5098718273904854
11
len  550
27.313162849618823 5.498581950727905 16.494308571006542
12
len  550
25.110746798585524 5.489951370108556 15.20093767373018
13
len  550
18.625197730806136 4.934271601632777 11.443651472124714
14
len  550
6.340039911219983 4.574249072098849 3.946510769224517
15
l

len  550
9.23389714278554 5.436124439833742 4.967152454489039
123
len  550
8.814899608398592 5.4409133951979 5.006566716371616
124
len  550
6.055610422577424 5.775104770181348 3.6304650754455388
125
len  550
18.194492019427265 5.296593019979491 10.218424878899782
126
len  550
6.14274299625573 5.305865893033475 3.656701859691573
127
len  550
35.41085088941378 5.297819721968222 20.908114560978078
128
len  550
10.078570936745509 4.973726602718269 5.798093407255749
129
len  550
1.1355243569047584 4.701196706220839 0.6614121179682932
130
len  550
1.1125742249543864 4.454792671786978 0.669332110749818
131
len  550
1.0957951726722324 4.708917655688023 0.6682357164860573
132
len  550
1.061387255865214 4.302220203157131 0.6814751259956875
133
len  550
0.7531360135737414 3.058477767951236 0.5105561658210007
134
len  550
3.2276498287998696 3.1706463748099494 1.8932790316033703
135
len  550
4.457916163209719 1.8562828149393156 2.256113118534657
136
len  550
1.8746730117134753 2.5682257598722043 1.